In [ ]:
## Step1 THUCNews 子集

In [2]:
import os
import glob
import pandas as pd

data_dir = "THUCNews1"
selected_labels = ["财经", "教育", "科技", "娱乐", "体育"]
samples_per_class = 2000
random_seed = 42

records = []

for label in selected_labels:
    class_path = os.path.join(data_dir, label)
    txt_files = glob.glob(os.path.join(class_path, "*.txt"))

    print(f"读取 {label} 类，共 {len(txt_files)} 文件")

    for fp in txt_files:
        try:
            with open(fp, "r", encoding="utf-8") as f:
                text = f.read().strip()
            if text:
                records.append({"text": text, "label": label})
        except Exception as e:
            print(f"读取失败：{fp}，错误：{e}")

print("总记录数：", len(records))
print("前几条：", records[:2])

# 如果 records 是空的，就报错退出
if not records:
    raise ValueError("❌ 没有读取到任何有效文本，请检查路径是否正确，以及是否是 UTF-8 编码")

df = pd.DataFrame(records)
print("DataFrame 列名：", df.columns.tolist())

# 去空、去重
df.dropna(subset=["text"], inplace=True)
df.drop_duplicates(subset=["text"], inplace=True)

# 每类采样
df_sampled = (
    df.groupby("label", group_keys=False)
      .apply(lambda x: x.sample(n=samples_per_class, random_state=random_seed))
      .reset_index(drop=True)
)

# 保存
df_sampled.to_json("THUCNews5类2000条.jsonl", orient="records", lines=True, force_ascii=False)
print("✅ 已保存为 JSONL 格式")


读取 财经 类，共 37098 文件
读取 教育 类，共 41936 文件
读取 科技 类，共 162929 文件
读取 娱乐 类，共 90396 文件
读取 体育 类，共 131604 文件
总记录数： 463962
前几条： [{'text': '新增资金入场 沪胶强势创年内新高\n\u3000\u3000记者 魏曙光\n\u3000\u3000本报讯 上海天然橡胶期价周三再创年内新高，主力合约突破21000元/吨重要关口。分析师指出，由于橡胶现货需求强劲，但供应却因主产国降雨天气而紧俏。同时国内有望出台新汽车刺激方案，沪胶后市有望延续强势。\n\u3000\u3000经过两个交易日的强势调整后，昨日上海天然橡胶期货价格再度大幅上扬，在收盘前1小时，大量场外资金涌入，主力1003合约强劲飙升很快升穿21000 元/吨整数关口，终盘报收于21,400元/吨，上涨2.27%，较前一日结算价上涨475元/吨，成交量为736,816手，持仓量为225,046 手。当日整体市场增仓3.4万余手。\n\u3000\u3000从盘后交易所持仓来看，两大主力多头金鹏期货和成都倍特期货略微增几百手，继续保持多头前两名位置，而主力多头新湖期货增仓3344手，值得注意的是，永安期货昨日空翻多，增加多仓1837手，其多头持仓增加至7021手，而净持仓增加至1813 手；空头两大主力则继续大幅增仓，其中浙江大地增仓2522手至17294手，银河期货增仓1075手至7086手。\n\u3000\u3000与此同时，东京商品交易所橡胶期货也强势上扬，基准4月合约再创13个月新高。截止北京时间昨日下午16点46分报241.5日元/公斤，较前日收盘涨3.2日元。\n\u3000\u3000金鹏期货北京海鹰路营业部总经理陈旭指出，近期沪胶受资金推动持续升创年内新高，而橡胶现货需求强劲，但供应却因主产国降雨天气而紧张。同时国内有望出台新汽车刺激方案，因此沪胶后市有望延续强势。\n\u3000\u3000泰国橡胶协会秘书长Prapas Euanontat16日表示，因暴雨中断生产，2009年该国橡胶产量可能下降约10%，为270万吨至280万吨。另据日本橡胶贸易协会最新数据，截至11月10日，该国天然橡胶库存较10月31日时的库存量下滑3.4%，至3902吨，创纪录新低。\n\u3000\u3000据

In [ ]:
用 SentencePiece / Byte-Pair Encoding 训练中英文混合子词
- 对比 vocab-size 对性能的影响	子词模型原理 & 超参调优

In [ ]:
一、子词模型原理概述
Byte-Pair Encoding (BPE)

基于“最频对合并”思想，从字符级开始，迭代地将出现频率最高的相邻字符对合并为一个新符号。

优点：训练速度快、易实现、对低频词有较好分割；

缺点：对未知字符无内置概率，分割较“刚性”。

SentencePiece（Unigram）

基于概率模型（Unigram language model），先从大词表抽取候选子词，再通过 EM 算法学习每个子词的概率，删减概率较低的子词。

优点：分割更灵活，可处理噪音字符；

缺点：训练相对慢，对大词表敏感。

中英文混合

中文无需空格分词，直接按字符或 n-gram 统计；英文按普通 BPE/Unigram 处理。

建议先做“预归一化”：所有字母转小写、保留数字/英文符号，中文字符按 Unicode 保留。

In [3]:
import jsonlines 

with jsonlines.open("THUCNews5类2000条.jsonl") as reader , open("all_texts.txt" , "w" , encoding = "utf-8") as fw:
    for obj in reader:
        fw.write(obj["text"].replace("\n" , " ") + "\n")